<a href="https://colab.research.google.com/github/ymiftah/operations_research/blob/master/lpQueens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qq pyomo
!apt-get install -qq coinor-cbc

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# The 8 Queens problem

## Problem Statement

8 Queens can be positioned on a chess board such as no queen is under threat of another

**Formulate the problem as a LP problem**

In [0]:
positions =  [i+str(j)
              for i in list('ABCDEFGH')
              for j in range(1,9)]
columns = [[i+str(j)
              for i in list('ABCDEFGH')]
              for j in range(1,9)]
rows = [[i+str(j)
              for j in range(1,9)]
              for i in list('ABCDEFGH')]
matrix= np.array(positions).reshape(8,8)
diagonals = [list(matrix.diagonal(i)) for i in range(-7,8)]
matrix = np.fliplr(matrix)
antidiag = [list(matrix.diagonal(i)) for i in range(-7,8)]

In [0]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# Sets
model.board = Set(initialize=positions)

# declare decision variables
model.queens = Var(model.board, domain=Binary)

# declare objective
model.obj = Objective(expr = sum(model.queens[p] for p in model.board),
                      sense=maximize)

#declare constraints
def cstr_queens(m):
    # Each task is assigned to exactly one cpu
    return (sum(m.queens[p] for p in m.board) == 8)
model.cstr_queens = Constraint(rule=cstr_queens)

i=0
for l in [rows, columns, diagonals, antidiag]:
    for pos in l:
        i+=1
        def cstr_rows(m):
            return (sum(m.queens[p] for p in m.board if p in pos) <= 1)
        setattr(model, 'cstr_%d'%i, Constraint(rule=cstr_rows))

In [6]:
# Solve the problem
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 8.0, 'Upper bound': 8.0, 'Number of objectives': 1, 'Number of constraints': 43, 'Number of variables': 64, 'Number of binary variables': 64, 'Number of integer variables': 64, 'Number of nonzeros': 64, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.02, 'Wallclock time': 0.02, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.037586212158203125}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Results

Let us visualize in a table the queens positions

In [8]:
assignment = model.queens.extract_values()
np.array(list(assignment.values())).reshape(8,8)

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.]])